# Operaciones sobre `RDD`s

Como se mencionó en la presentación, hay dos tipos de operaciones sobre los `RDD`s: _transformaciones_ y _acciones_.

- Las _transformaciones_ construyen un `RDD` nuevo a partir del anterior.
  - Cada transformación queda guardada por =Spark= en el /lineage graph/ un *DAG*.

- Las _acciones_ calculan un resultado basado en el `RDD`.

- La diferencia es que las `RDD` son computadas en forma _lazy_, sólo son ejecutadas hasta la acción.

- Si quieres usarlo una `RDD` varias veces debes de persistirla (con `persist()`).

## Flujo típico de trabajo

1. Crear un `RDD` a partir de datos externos.
2. Transformarlo a nuevos `RDDs`.
3. Persistir algunos `RDDs` para su uso posterior.
4. Lanzar acciones.

Obtenemos el `SparkContext` para poder trabajar

In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

## Transformaciones

Las principales transformaciones (o por lo menos las más usadas) se listan a continuación

  - `map`
    - Usa una función y la aplica a cada elemento del `RDD`, el resultado se guarda en un nuevo `RDD`.
  - `filter`
    - Usa una función y devuelve sólo los elementos que pasan la función (que devuelven verdadero) en el nuevo `RDD`.
  - `flatMap`
    - Como el `map` pero regresa un iterador por cada elemento
      - Por ejemplo una función que divide una cadena.
  - `distinct`
  - `sample`
  - `join`
  - `cogroup`
  - `coalesce`
  - `union`, `intersection`, `substract`, `cartesian`


**NOTA** En los ejemplos que siguen usaremos `collect()`,  `count()`, `take()`. Estas funciones **no** son _transformaciones_, sino _acciones_ que se explican más abajo.

Creamos un `RDD` a partir de enteros (justo como antes)

In [ ]:
numeros = sc.parallelize(range(1000))

Usaremos una _función anónima_ para elevar los números al cuadrado

In [ ]:
cuadrados = numeros.map(lambda x: x*x)

In [ ]:
cuadrados.take(5)

In [ ]:
muestra = cuadrados.sample(fraction=0.3, withReplacement=False)

In [ ]:
muestra.count()

In [ ]:
muestra.take(5)

In [ ]:
pares = muestra.filter(lambda x: x%2 == 0)

In [ ]:
pares.take(5)

In [ ]:
pares.count()

El `DAG` está formado por `numeros -> cuadrados -> muestra -> pares` 

Estar verificando en cada paso, no es muy eficiente, de hecho, una manera de programar muy utilizada es la siguiente:

In [ ]:
pares2 = numeros.map(lambda x: x*x)\
                .sample(fraction=0.3, withReplacement=False)\
                .filter(lambda x: x%2 == 0)

In [ ]:
pares2.take(5)

In [ ]:
pares2.count()

El `DAG` tiene la misma estructura (con `numeros` como raíz), pero sus nodos son anónimos.

Una transformación que causa confusión es `flatMap`, veamos un ejemplo

La función `.split()` de `python`, toma una cadena y devuelve una lista

In [ ]:
"Hola a todos".split(" ")

¿Qué efectos tendría en un `map`?

In [ ]:
frases = sc.parallelize(["hola a todos", "taller nacional de big data", "Análisis de redes sociales"])

In [ ]:
palabras = frases.map(lambda frase: frase.split(" ")).collect()
palabras

In [ ]:
palabras[1]

Obtenemos un arreglo de arreglos y quizá esto no sea lo que necesitamos. Usando `flatMap` "aplanamos" el `RDD` resultante.

In [ ]:
palabras = frases.flatMap(lambda frase: frase.split(" ")).collect()
palabras

## Acciones

- `first`
- `take`, `takeSample`
- `reduce`
  - Opera en dos elementos del mismo tipo del `RDD` y regresa un elemento del mismo tipo.
- `aggregate`
  - Nos permite implementar acumuladores.
- `collect`
  - Regresa el `RDD` completo.
- `count`, `countByValue`, `top`, `foreach`, `countByKey`
- `saveAsTextFile`


Es importante notar que todos estas operaciones acaban con datos en el _driver_.

In [ ]:
numeros.first()

In [ ]:
numeros.take(5)

In [ ]:
numeros.takeSample(num=30, withReplacement=False)

In [ ]:
suma = numeros.reduce(lambda x, y: x + y)
suma

In [ ]:
pares.top(10)

Para los ejemplos que siguen generaremos un conjunto falso de transacciones, usando las bibliotecas de `python` `random` y `uuid`

In [ ]:
import random

In [ ]:
random.randint(10,1000)

In [ ]:
accion = ['RETIRO', 'COMPRA', 'CONSULTA']
random.choice(accion)

In [ ]:
import uuid

In [ ]:
clientes = [str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4()), str(uuid.uuid4())]

In [ ]:
clientes

In [ ]:
def generate_transaction():
    """
    Regresa una transacción falsa, la primera columna es el número de tarjeta ofuscado, las demás
    columnas son el comercio, la acción realizada en el comercio y el monto de la acción.
    Devuelve una cadena separada por pipes (|)
    """
    comercio = ['ARENA COLISEO', 'SUPERCITO', 'RESTAURANTE EL TRABAJO']
    accion = ['RETIRO', 'COMPRA']
    
    return "%s|%s|%s|%s" % (random.choice(clientes), random.choice(comercio), random.choice(accion), random.randint(10, 10000))

In [ ]:
?generate_transaction

In [ ]:
generate_transaction()

In [ ]:
def generate_transactions(number=10000):
    """
    Regresa una lista de transacciones falsa.
    """
    txs = []
    for i in range(number):
        txs.append(generate_transaction())
    return txs

In [ ]:
generate_transactions(number=10)

In [ ]:
txs = sc.parallelize(generate_transactions(number=10000))

In [ ]:
txs.first()

In [ ]:
txs.count()

Guardamos estas transacciones para usarlas posteriormente

In [ ]:
! rm -R output/raw/transacciones

In [ ]:
txs.saveAsTextFile("output/raw/transacciones")

Como está distribuido, los archivos en realidad se guardan como carpeta.

In [ ]:
! ls -lh output/raw/transacciones

Supongamos que queremos realizar un conteo por tarjeta, los pasos serían los siguientes:

Designamos el número de tarjeta como la **llave** (_key_)

In [ ]:
kv_txs = txs.map(lambda x: x.split("|"))\
            .map(lambda x: (x[0], x[1:])) # x[0] contiene el número de tarjeta ofuscado
kv_txs.take(5)

In [ ]:
kv_txs.keys().first()

In [ ]:
kv_txs.values().first()

In [ ]:
kv_txs.count()

In [ ]:
kv_txs.countByKey()